In [222]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import pyodbc
from globalfunctions import *
%matplotlib inline

Ticker = 'EDC'
upperChannel = 20
lowerChannel = 50
model = str(upperChannel)+'-'+str(lowerChannel) +'-DC'
fileName = Ticker + '.csv'
df = pd.read_csv(fileName)

df.info()    


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2852 entries, 0 to 2851
Data columns (total 7 columns):
Date         2852 non-null object
Open         2852 non-null float64
High         2852 non-null float64
Low          2852 non-null float64
Close        2852 non-null float64
Adj Close    2852 non-null float64
Volume       2852 non-null int64
dtypes: float64(5), int64(1), object(1)
memory usage: 156.1+ KB


In [127]:
df = df.dropna()
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
3806,2020-04-23,35.549999,35.770000,35.139999,35.139999,35.139999,19398700
3807,2020-04-24,35.209999,35.270000,34.790001,35.090000,35.090000,15272300
3808,2020-04-27,35.509998,35.810001,35.470001,35.750000,35.750000,17169500
3809,2020-04-28,36.259998,36.299999,35.790001,35.840000,35.840000,15768600
3810,2020-04-29,36.430000,36.889999,36.369999,36.810001,36.810001,14100100


In [157]:
def donchianChannelAI(nHigh,nLow,n,positions):
    posCount = len(positions)
    if(posCount == 0):
       #No position history exist
        if(n['Close'] >= nHigh):
            #Opening first position!
            newPosition = Position(n['Close'],n['Date'],0,'01/01/1900',0, 'Long')
            return [1,newPosition]
        else:
            #Entry criteria not met
            return [0,None]
    else:
        #Positions exist, get latest position
        currentPosition = positions[posCount-1]
        
        #latest position is still open
        if(currentPosition.exitPrice ==0):  
            #Check if close is below recent low
            if(n['Close'] < nLow):
                #price closed below recent low, update current position
                currentPosition.exitPrice = n['Close']
                currentPosition.exitDate = n['Date']
                currentPosition.profitLoss = n['Close']/currentPosition.entryPrice
                
                #return closed position to update latest item in 'Positions'
                return [2,currentPosition]
            else:
                #price closed above recent low, return null action
                return [0,None]
        else:
            #Latest position has been closed, check if new position is needed
            if(n['Close'] >= nHigh):
                newPosition = Position(n['Close'],n['Date'],0,'01/01/1900',0, 'Long')
                return [1,newPosition]
            else:
                return [0,None]
    


In [225]:
def donchianChannelPositions(upperChannel,lowerChannel):
    highs = []
    lows = []
    nHigh = 0
    nLow = 0
    i = 0
    positions = []


    for j,n in df.iterrows():
        highs.append(n['High'])
        lows.append(n['Low'])
        i +=1

        if i >= upperChannel:
            #highs.remove(highs[0])
            highs.pop(0)
        if i >= lowerChannel:
            #lows.remove(lows[0])
            lows.pop(0)
        if i >=upperChannel:
            pos = donchianChannelAI(nHigh,nLow,n,positions)

            if(pos[0]==1):
                #new position to add
                positions.append(pos[1])
            elif(pos[0]==2):
                #drop most recent position and replace with updated version
                positions.pop()
                positions.append(pos[1])

        nHigh = max(highs)
        nLow = min(lows)
    return positions


In [226]:
#labels = ['EntryDate','EntryPrice','ExitDate','ExitPrice','PLPercent','Ticker','Model']
pos = donchianChannelPositions(upperChannel,lowerChannel)
ModelAssessment(pos,Ticker,model)

                   EDC   20-50-DC
0    Total Return (TR)   2.065596
1     Geometric Return   0.036937
2                 Mean   0.107840
3                StDev   0.490459
4                  Min  -0.312659
5               Median  -0.037395
6                  Max   1.879791
7                    N  20.000000
8  Bootstrap Median TR   1.952051
9   Bootstrap Stdev TR  52.256841


In [271]:

#create(positions,Ticker,model)sqlStatement,params = []
def read(sSQL,sParams = []):
    #Establish Connection to SQL DB
    conn = pyodbc.connect(
    "Driver={SQL Server};"
    "Server=DESKTOP-E4GJV84\SQLEXPRESS;"
    "Database=QuantAnalysis;"
    "Trusted_Connection=yes;"
    )
    
    df = pd.read_sql_query(sSQL,conn,params=tuple(sParams))
    
    return df

    

In [273]:
sSQL = "with cte_one as( Select Top 10 Ticker, percentFromHigh, 'shares' = cast(round(?/closePrice,2) as decimal(5,2)), closePrice From [QuantAnalysis].[dbo].[tblStockPerformance] Order by percentFromHigh desc) Select a.ticker, a.shares,'estimatedCost' = a.shares * a.closePrice  From cte_one a where a.percentFromHigh >= (Select Min(percentFromHigh) From cte_one where ticker in ('SHY','IEF','TLT','GLD'))"
df = read(sSQL,[1000.00])

df.head()

,ticker,shares,estimatedCost
0,SHY,11.54,1000.170646
1,CDNS,12.22,1000.207000
2,DXCM,2.94,1001.628600
3,IEF,8.20,999.785000
4,BND,11.42,1000.163600
